# LMC中YSOs的IFU

**用例：** 自动检测点源并提取光度数据于三维立方体中。分析光谱线。<br>

**数据：** ALMA 13CO 数据立方体。<br>

**工具：** specutils, photutils, astropy。<br>

**跨仪器：** NIRSpec, MIRI。<br>

**文档：** 本笔记本是STScI更大[后处理数据分析工具生态系统](https://jwst-docs.stsci.edu/jwst-post-pipeline-data-analysis)的一部分，可以直接从[JDAT笔记本GitHub目录](https://github.com/spacetelescope/jdat_notebooks)中[下载](https://github.com/spacetelescope/dat_pyinthesky/tree/main/jdat_notebooks/MRS_Mstar_analysis)。<br>

LMC是研究大质量恒星形成的理想场所，因为它相对较近（距离约50 kpc），并且呈面向观测者的方向，使我们能够解析单个恒星形成区域，消除视线混淆。斯皮策（Spitzer）和赫歇尔（Herschel）调查通过从3微米到500微米的光度测量，识别了LMC中数千个年轻恒星对象（YSOs），并且对8个太阳质量的YSOs的探测是完整的。

我们知道，大质量恒星形成发生在密集的分子气体中。阿尔玛（ALMA）对12CO和13CO分子气体的观测显示，碰撞的细丝可能是使气体密度高达10^6 cm^-3的机制。在阿尔玛观测到的碰撞CO气体细丝的中心，观察到了质量高达50个太阳质量的大质量恒星。

詹姆斯·韦伯太空望远镜（JWST）上的仪器将具有比斯皮策太空望远镜高10倍的角分辨率和超过当前仪器百倍的灵敏度。使用MIRI和NIRCam对YSOs的观测将完整到1个太阳质量的YSOs，而斯皮策观测只能完整到8个太阳质量的YSOs。LMC中的大质量YSOs很可能是一个小的YSO簇，借助MRS我们可以获取簇中单个对象的光谱。这将成为理解恒星形成和星团形成的强大工具。

在第一部分中，我使用ALMA的13CO数据立方体读取数据并制作图形。在这个笔记本中显示的区域内有近400颗大质量恒星。这个笔记本的目标之一是使用photutils自动检测点源并在给定的3D立方体中提取光度。第二部分中，我使用斯皮策/IRS的已知早期阶段YSOs的光谱。YSOs具有特定的吸收和发射线。目标是首先使用specutils找到重要的谱线，以确定某个对象是否为YSO，基于存在或不存在的谱线。接下来的步骤是根据检测到的发射和吸收线，识别YSO是阶段I（大多数被CO2冰特征嵌入）、阶段II（更进化，具有冰和硅酸盐吸收特征）还是阶段III（几乎是平坦的光谱，带有PAH发射）。

### 导入所需的内容

In [ ]:
import os  # 导入操作系统模块

from astropy import units as u  # 导入Astropy单位模块
from astropy.wcs import WCS  # 导入天文坐标系统模块
from astropy.io import ascii  # 导入ASCII输入输出模块
from astropy.nddata import StdDevUncertainty  # 导入标准偏差不确定性模块
from astropy.table import Table  # 导入表格模块
from astropy.stats import sigma_clipped_stats  # 导入sigma裁剪统计模块
import urllib.request  # 导入urllib请求模块

import numpy as np  # 导入NumPy模块

from spectral_cube import SpectralCube  # 导入光谱立方体模块
from photutils.detection import DAOStarFinder  # 导入DAO星体查找器模块
from photutils.aperture import CircularAperture  # 导入圆形光圈模块

from specutils import Spectrum1D, SpectralRegion  # 导入光谱1D和光谱区域模块
from specutils.analysis import snr  # 导入信噪比分析模块
from specutils.fitting import fit_generic_continuum, find_lines_derivative  # 导入拟合通用连续谱和查找线的导数模块

In [ ]:
%matplotlib inline  # 在Jupyter Notebook中内联显示图形

import matplotlib.pyplot as plt  # 导入matplotlib库中的pyplot模块，用于绘图

### 检查导入内容的版本

In [ ]:
import aplpy  # 导入aplpy库，用于天文数据可视化

from aplpy import __version__ as aplpy_version  # 获取aplpy库的版本信息

print("Aplpy: {}".format(aplpy_version))  # 打印aplpy库的版本信息

from astrodendro import __version__ as astrodendro_version  # 导入astrodendro库并获取版本信息

print("Astrodendro: {}".format(astrodendro_version))  # 打印astrodendro库的版本信息

from astropy import __version__ as astropy_version  # 导入astropy库并获取版本信息

print("Astropy: {}".format(astropy_version))  # 打印astropy库的版本信息

from jwst import __version__ as jwst_version  # 导入jwst库并获取版本信息

print("JWST: {}".format(jwst_version))  # 打印JWST库的版本信息

from matplotlib import __version__ as matplotlib_version  # 导入matplotlib库并获取版本信息

print("Matplotlib: {}".format(matplotlib_version))  # 打印matplotlib库的版本信息

import numpy  # 导入numpy库，用于数值计算

from numpy import __version__ as numpy_version  # 获取numpy库的版本信息

print("Numpy: {}".format(numpy_version))  # 打印numpy库的版本信息

from pandas import __version__ as pandas_version  # 导入pandas库并获取版本信息

print("Pandas: {}".format(pandas_version))  # 打印pandas库的版本信息

from photutils import __version__ as photutils_version  # 导入photutils库并获取版本信息

print("Photutils: {}".format(photutils_version))  # 打印photutils库的版本信息

from scipy import __version__ as scipy_version  # 导入scipy库并获取版本信息

print("Scipy: {}".format(scipy_version))  # 打印scipy库的版本信息

from specutils import __version__ as specutils_version  # 导入specutils库并获取版本信息

print("Specutils: {}".format(specutils_version))  # 打印specutils库的版本信息

from spectral_cube import __version__ as spectral_cube_version  # 导入spectral_cube库并获取版本信息

print("SpectralCube: {}".format(spectral_cube_version))  # 打印spectral_cube库的版本信息

### 设置输出目录以存放图形和光谱

用户需要设置他们希望输出文件存放的路径。

In [ ]:
import os  # 导入os模块，用于处理文件和目录

output_images = './Images'  # 输出图像的存放路径
output_spectra = './Spectra'  # 输出光谱的存放路径

# 遍历输出图像和光谱的目录列表
for directory in [output_images, output_spectra]:
    
    # 检查目录是否存在
    if not os.path.exists(directory):
        os.makedirs(directory)  # 如果不存在，则创建该目录

### 设置绘图参数

In [ ]:
# 定义绘图参数的字典
params = {
    'legend.fontsize': '18',  # 图例字体大小
    'axes.labelsize': '18',   # 坐标轴标签字体大小
    'axes.titlesize': '18',   # 坐标轴标题字体大小
    'xtick.labelsize': '18',   # x轴刻度标签字体大小
    'ytick.labelsize': '18',   # y轴刻度标签字体大小
    'lines.linewidth': 2,      # 线条宽度
    'axes.linewidth': 2,       # 坐标轴线宽度
    'animation.html': 'html5'  # 动画输出格式
}

# 更新绘图参数
plt.rcParams.update(params)

# 更新最大打开警告参数为0
plt.rcParams.update({'figure.max_open_warning': 0})

### 设置数据路径

使用来自大麦哲伦云（LMC）中一个已知星形成区域的ALMA 13CO数据，该区域拥有近150个大质量年轻恒星（YSOs）。

#### 请注意，数据文件大小约为2 GB，因此下载时间将根据您的互联网速度而有所不同。您也可以直接使用下面的链接进行下载。

In [ ]:
import os  # 导入os模块，用于文件和目录操作
import urllib.request  # 导入urllib.request模块，用于下载文件

data_file = './LMC_13CO.fits'  # 定义数据文件的路径

# 检查文件是否存在
if os.path.exists("LMC_13CO.fits"):  # 如果文件存在
    print("LMC_13CO.fits Already Exists")  # 打印文件已存在的消息
else:  # 如果文件不存在
    print("Downloading LMC_13CO.fits")  # 打印正在下载的消息

    # 定义文件下载的URL
    url = 'https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/MIRI_IFU_YSOs_in_the_LMC/LMC_13CO.fits'

    # 下载文件并保存到指定路径
    urllib.request.urlretrieve(url, './LMC_13CO.fits')  # 从URL下载文件并保存为LMC_13CO.fits

### 加载并显示数据立方体

In [ ]:
# 从指定的文件中读取光谱立方体数据，hdu=1表示读取第一个数据单元
cube = SpectralCube.read(data_file, hdu=1)  

# 打印光谱立方体的基本信息
print(cube)

### 修剪立方体

不确定JWST MRS立方体的情况，但在ALMA中，立方体的开始和结束部分数据质量较差，因此如果需要，可以裁剪立方体以减小其大小。在这个例子中，我使用了整个立方体。

如果您想将立方体裁剪到特定的光谱范围，请执行以下操作：

subcube = cube.spectral_slab(240 * (u.m/u.s), 265 * (u.m/u.s))

cube = subcube

In [ ]:
# 允许进行大规模操作
cube.allow_huge_operations = True

# 获取立方体的最小子立方体
cmin = cube.minimal_subcube()

### 从3D立方体生成2D图像，方法可以是平均值、中央値或总和

这一步是为了寻找点源。在定位点源后，您将应用孔径光度法（aperture photometry）来提取光谱。

**代码可能需要一些时间**，因为数据立方体很大。

In [ ]:
cont_img = cmin.sum(axis=0)  # 对cmin数组在第0轴（行）上进行求和，生成一个新的二维图像cont_img

### 绘制图像以进行视觉检查

这是为了查看您是否能看到任何发射，并确保您拥有正确的数据立方体。

In [ ]:
fig = plt.figure()  # 创建一个新的图形对象

plt.imshow(cont_img.value)  # 显示图像数据，cont_img.value包含图像的像素值

plt.tight_layout()  # 自动调整子图参数，以给图形的各个部分留出足够的空间

plt.show()  # 显示图形

### 绘制带有WCS坐标的合成图像并保存图形

In [ ]:
name = '13CO'  # 定义变量name，赋值为'13CO'

In [ ]:
F = aplpy.FITSFigure(cont_img.hdu, north=True)  # 创建FITSFigure对象，设置北方向

F.show_colorscale()  # 显示颜色比例尺

F.add_label(0.1, 0.9, name, relative=True, size=22, weight='bold')  # 添加标签，位置相对，字体大小和加粗设置

F.axis_labels.set_font(size=22)  # 设置坐标轴标签的字体大小

F.tick_labels.set_font(size=18, stretch='condensed')  # 设置刻度标签的字体大小和压缩样式

F.save(output_images+"_"+name+".pdf", dpi=300)  # 保存图像为PDF格式，设置分辨率为300 dpi

### 使用 photutils 识别所有点源

In [ ]:
# 空数组用于存储值
name_val = []  # 存储对象名称
source_val = []  # 存储源值
ra_val = []  # 存储右升角值
dec_val = []  # 存储 declination 值

# 计算累积图像的均值、中位数和标准差
mean, median, std = sigma_clipped_stats(cont_img.value, sigma=2.0)  # 使用 sigma 裁剪统计方法计算均值、中位数和标准差

### 获取所有点源的列表

请注意，通常使用 `3*std` 来寻找高于噪声水平的源，但我这样做时发现有 247 个点源。因此，我将其调整为 `6*std`，以查看 4 个点源并确保此步骤有效。

In [ ]:
# 创建DAOStarFinder对象，设置全宽半最大值（fwhm）和阈值
daofind = DAOStarFinder(fwhm=2.0, threshold=6*std)

# 在减去中位数后的图像中寻找星源
sources = daofind(cont_img.value - median) 

# 打印场中源的数量
print("\n  Number of sources in field: ", len(sources))

### 提取并绘制所有源的光谱

**此步骤可能需要一些时间来运行。**

进行视觉检查，以确认所有点源是否已被识别，尽管最终结果应该是不需要这样做。此外，作为科学用户，我希望算法能够从数学上告诉我什么是点源，什么不是。应该有一些标准来区分点源和扩展源，用户不应该需要进行视觉检查。在这四个源中，有两个是噪声光谱，因为它们检测到了ALMA立方体边界上的噪声峰值，这是ALMA数据立方体中常见的现象。

In [ ]:
# 使表格与RA和DEC坐标一致

if len(sources) > 0:  # 如果sources列表不为空

    for col in sources.colnames:  # 遍历sources的每一列

        sources[col].info.format = '%.8g'  # 设置每列的显示格式为8位有效数字

    print(sources)  # 打印sources表格

    # 将xcentroid和ycentroid转换为RA和DEC坐标

    positions = Table([sources['xcentroid'], sources['ycentroid']])  # 创建包含x和y中心的表格

    w = WCS(cont_img.header)  # 创建WCS对象以转换坐标

    radec_lst = w.pixel_to_world(sources['xcentroid'], sources['ycentroid'])  # 将像素坐标转换为世界坐标

    # 使用圆形光圈提取点源的光谱

    for countS, _ in enumerate(sources):  # 遍历每个源

        print(radec_lst[countS].to_string('hmsdms'))  # 以hms dms格式打印RA和Dec值

        name_val.append(name)  # 将名称添加到name_val列表

        source_val.append(countS)  # 将源索引添加到source_val列表

        ra_val.append(radec_lst[countS].ra.deg)  # 将RA值（度）添加到ra_val列表

        dec_val.append(radec_lst[countS].dec.deg)  # 将DEC值（度）添加到dec_val列表

        # 帧的大小

        ysize_pix = cmin.shape[1]  # 获取y轴像素大小

        xsize_pix = cmin.shape[2]  # 获取x轴像素大小

        # 为源设置一些质心像素

        ycent_pix = sources['ycentroid'][countS]  # 获取y中心像素

        xcent_pix = sources['xcentroid'][countS]  # 获取x中心像素

        # 为源创建一个光圈半径

        # 这可以是用户根据自己的科学专业知识输入的内容。

        apertureRad_pix = 2  # 设置光圈半径为2像素

        # 为光圈创建一个掩膜数组

        yy, xx = np.indices([ysize_pix, xsize_pix], dtype='float')  # 检查ycentpix和xcentpix的顺序是否正确

        radius = ((yy - ycent_pix)**2 + (xx - xcent_pix)**2)**0.5  # 在帧中创建一个圆形

        mask = radius <= apertureRad_pix  # 选择光圈半径内的像素

        maskedcube = cmin.with_mask(mask)  # 创建一个掩膜立方体

        pixInAp = np.count_nonzero(mask == 1)  # 计算光圈内的像素数量

        spectrum = maskedcube.sum(axis=(1, 2))  # 从仅包含光圈的区域提取光谱

        noisespectrum = maskedcube.std(axis=(1, 2))  # 提取源的噪声光谱

        # 从背景测量光谱。使用围绕源的光环。

        an_mask = (radius > apertureRad_pix + 1) & (radius <= apertureRad_pix + 2)  # 选择光环内的像素

        an_maskedcube = cmin.with_mask(an_mask)  # 创建一个掩膜立方体

        # 绘制从圆形光圈提取的光谱：通过求和提取

        fig = plt.figure(figsize=(10, 5))  # 创建一个图形窗口

        plt.plot(maskedcube.spectral_axis.value, spectrum.value, label='Source')  # 绘制源光谱

        plt.xlabel('Frequency [Hz]')  # 设置x轴标签为频率

        plt.ylabel('Kelvin')  # 设置y轴标签为开尔文

        plt.gcf().text(0.5, 0.85, name, fontsize=14, ha='center')  # 在图形上方添加源名称

        plt.gcf().text(0.5, 0.80, radec_lst[countS].to_string('decimal'), ha='center', fontsize=14)  # 添加RA和DEC值

        plt.legend(frameon=False, fontsize='medium')  # 添加图例

        plt.tight_layout()  # 调整布局以避免重叠

        plt.show()  # 显示图形

        plt.close()  # 关闭图形

        positions_pix = (sources['xcentroid'], sources['ycentroid'])  # 获取像素位置

        apertures = CircularAperture([positions_pix[0][countS], positions_pix[1][countS]], r=2.)  # 创建圆形光圈对象

        fig = plt.figure()  # 创建新图形

        plt.subplot(1, 2, 1)  # 创建1行2列的子图，选择第一个子图

        plt.imshow(cont_img.value, cmap='Greys', origin='lower')  # 显示背景图像

        apertures.plot(color='blue', lw=1.5, alpha=0.5)  # 绘制光圈

        plt.subplot(1, 2, 2)  # 选择第二个子图

        plt.imshow(cont_img.value, origin='lower')  # 显示背景图像

        plt.tight_layout()  # 调整布局以避免重叠

        plt.show()  # 显示图形

        plt.close()  # 关闭图形

### 制作提取源的表格

In [ ]:
# 创建一个包含天体信息的表格，包含名称、源编号、右升角和 declination
sourceExtSpecTab = Table([name_val, source_val, ra_val, dec_val], 

                         names=("name", "source_no", "ra", "dec"))

# 打印表格内容
print(sourceExtSpecTab)   

# 生成输出文件的完整路径，文件名为 'YSOsourcesSpec_list.csv'
os.path.join(output_spectra, 'YSOsourcesSpec_list.csv')

# 将表格数据写入 CSV 文件，允许覆盖同名文件
ascii.write(sourceExtSpecTab, os.path.join(output_spectra, 'YSOsourcesSpec_list.csv'), format='csv', overwrite=True)

### 从此使用斯皮策IRS YSO光谱进行科学测试案例

(1) 在光谱中寻找谱线。来自H₂O、NH₃、CH₃OH、HCOOH和H₂CO的5-7微米范围内的冰特征难以识别，因为与多环芳烃（PAH）存在混淆。15.2微米的CO₂冰吸收特征更易于识别。更为演化的年轻恒星对象（YSOs）将在6.2微米、7.7微米、8.6微米、11.3微米和12.7微米处具有PAH和精细结构特征。但PAH和精细结构可能意味着一个更为演化的H II区，而不是一个嵌入的YSO。YSOs预计会有H₂发射。光化学区（PDRs）和冲击波在YSO环境附近都会导致H₂发射。

(2) 更详细地观察冰特征、PAH特征和硅酸盐特征。

(3) 识别YSOs。

用户可以决定要检查哪个YSO，YSO1、YSO2或YSO3，并相应地注释掉代码。

In [ ]:
# 设置 YSO1 数据的路径
YSO1 = 'https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/MIRI_IFU_YSOs_in_the_LMC/yso1_108_spec.txt'

# 设置 YSO2 数据的路径
# YSO2 = 'https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/MIRI_IFU_YSOs_in_the_LMC/yso2_102_spec.txt'

# 设置 YSO3 数据的路径
# YSO3 = 'https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/MIRI_IFU_YSOs_in_the_LMC/yso3_4536_spec.txt'

在这一部分中也选择 YSO1、YSO2 或 YSO3。

In [ ]:
# 读取光谱数据并进行初步可视化检查

data = ascii.read(YSO1)  # 从YSO1文件中读取数据

# data = ascii.read(YSO2)  # 从YSO2文件中读取数据（注释掉）
# data = ascii.read(YSO3)  # 从YSO3文件中读取数据（注释掉）

# 检查数据的列名，如果第一列名为'col1'
if data.colnames[0] == 'col1':
    data['col1'].name = 'wave_mum'  # 将第一列重命名为'wavelength_mum'
    data['col2'].name = 'cSpec_Jy'   # 将第二列重命名为'cSpec_Jy'
    data['col3'].name = 'errFl_Jy'   # 将第三列重命名为'errFl_Jy'

# 将数据转换为适当的单位
wav = data['wave_mum'] * u.micron  # 波长：微米
fl = data['cSpec_Jy'] * u.Jy        # 流量密度：Jy
efl = data['errFl_Jy'] * u.Jy       # 流量误差：Jy

# 创建一个1D光谱对象
spec = Spectrum1D(spectral_axis=wav, flux=fl, uncertainty=StdDevUncertainty(efl))  

# 创建一个图形对象
fig = plt.figure(figsize=(8, 4))  
plt.plot(spec.spectral_axis, spec.flux, label='spectrum')  # 绘制光谱曲线

# 设置坐标轴标签
plt.xlabel('Wavelength (microns)')  
plt.ylabel("Flux ({:latex})".format(spec.flux.unit))  # 设置流量单位标签

plt.legend(frameon=False, fontsize='medium')  # 添加图例
plt.tight_layout()  # 调整布局以适应图形
plt.show()  # 显示图形
plt.close()  # 关闭图形

### 拟合通用连续谱

在拟合一个通用的连续谱（continuum）时，似乎在5微米到7微米以及17微米到25微米的范围内，连续谱被高估了，而没有排除任何波长。这导致了发射线（emission lines）和吸收线（absorption lines）的错误识别。相反，我在5-13微米范围内拟合了一个连续谱，并在13-35微米范围内拟合了另一个连续谱。这提供了一个更好的连续谱减去后的光谱（continuum subtracted spectrum）。但这意味着我需要跟踪两份不同的吸收和发射特征的线列表（line lists）。如果我能拟合一个连续谱，那将更好。

In [ ]:
# 计算信噪比
sig2noise = np.round(snr(spec), 2)  # 计算并四舍五入信噪比

# 为排除几个谱线，拟合波长大于13微米的一个连续谱
to_exclude_1 = [(5.0, 13.0)*u.micron, (14.5, 15.5) * u.micron, (17.0, 18.0) * u.micron]  # 定义要排除的谱线/区域
exclude_region_1 = SpectralRegion(to_exclude_1)  # 创建一个specutils区域

# 为排除硅酸盐吸收特征，拟合波长小于13微米的另一个连续谱
to_exclude_2 = [(7.0, 11.0)*u.micron, (13.0, 35.0)*u.micron]  # 定义要排除的谱线/区域
exclude_region_2 = SpectralRegion(to_exclude_2)  # 创建一个specutils区域

continuum_model_1 = fit_generic_continuum(spec, exclude_regions=exclude_region_1)  # 生成第一个连续谱模型
continuum_model_2 = fit_generic_continuum(spec, exclude_regions=exclude_region_2)  # 生成第二个连续谱模型
y_continuum_1 = continuum_model_1(spec.spectral_axis)  # 将第一个连续谱模型应用于1D光谱对象
y_continuum_2 = continuum_model_2(spec.spectral_axis)  # 将第二个连续谱模型应用于1D光谱对象

# 生成去除连续谱和归一化的光谱。两者都需要用于后续分析。
spec_norm_2 = spec / y_continuum_2  # 归一化光谱

count_axis = 0  # 初始化计数轴
for i in spec.spectral_axis:  # 遍历光谱轴
    if i.value < 13.0:  # 如果波长小于13微米
        count_axis = count_axis + 1  # 计数器加一

# 将变量赋值以提高可读性
spec_contsub_1 = spec[count_axis:len(spec.spectral_axis)] - y_continuum_1[count_axis:len(spec.spectral_axis)]  # 去除连续谱1
spec_contsub_2 = spec[0:count_axis] - y_continuum_2[0:count_axis]  # 去除连续谱2

In [ ]:
# 绘制连续谱和光谱

fig = plt.figure(figsize=(10, 6))  # 创建一个10x6英寸的图形

plt.plot(spec.spectral_axis, spec.flux, label='Source')  # 绘制源光谱

plt.plot(spec.spectral_axis, y_continuum_1, label='Continuum Fit to lambda > 13 microns')  # 绘制波长大于13微米的连续谱拟合

plt.plot(spec.spectral_axis, y_continuum_2, label='Continuum Fit to lambda < 13 microns')  # 绘制波长小于13微米的连续谱拟合

plt.xlabel("Wavelength ({:latex})".format(spec.spectral_axis.unit))  # 设置x轴标签为波长，并格式化单位

plt.ylabel("Flux ({:latex})".format(spec.flux.unit))  # 设置y轴标签为通量，并格式化单位

plt.legend(frameon=False, fontsize='medium')  # 添加图例，去掉边框，设置字体大小为中等

plt.tight_layout()  # 自动调整子图参数，使之填充整个图像区域

plt.show()  # 显示图形

plt.close()  # 关闭图形窗口

In [ ]:
# 绘制去除连续谱的光谱

fig = plt.figure(figsize=(10, 6))  # 创建一个10x6英寸的图形

plt.plot(spec_contsub_1.spectral_axis, spec_contsub_1.flux, color='black')  # 绘制第一个去除连续谱的光谱，颜色为黑色

plt.plot(spec_contsub_2.spectral_axis, spec_contsub_2.flux, color='black')  # 绘制第二个去除连续谱的光谱，颜色为黑色

plt.axhline(y=0.0, color='r', linestyle='-')  # 绘制y=0的红色水平线

plt.xlabel("Wavelength ({:latex})".format(spec.spectral_axis.unit))  # 设置x轴标签，显示波长单位

plt.ylabel("Flux ({:latex})".format(spec.flux.unit))  # 设置y轴标签，显示通量单位

plt.tight_layout()  # 调整子图参数，使之填充整个图像区域

plt.show()  # 显示图形

plt.close()  # 关闭图形窗口

### 寻找发射线和吸收线

常见的年轻恒星对象（YSO）谱线包括PAH发射线、冰吸收线和硅酸盐吸收线。

对于`find_lines_derivative()`函数，使用信噪比（SNR）阈值会比使用通量阈值更好。

用户需要根据他们正在检查的YSO（YSO1、YSO2或YSO3）来移除相应的注释。如果使用的是SNR阈值，那么这个过程可以更自动化一些。

In [ ]:
# 在连续体减去的光谱中查找发射线和吸收线。

# 这将打印出两个发射线列表和两个吸收线列表。

if sig2noise > 10:  # 如果信噪比大于10

    lines = find_lines_derivative(spec_contsub_1, flux_threshold=0.020)  # 查找YSO 1的发射和吸收线

    # lines = find_lines_derivative(spec_contsub_1, flux_threshold=0.03)  # 查找YSO 2的发射和吸收线

    # lines = find_lines_derivative(spec_contsub_1, flux_threshold=0.02)  # 查找YSO 3的发射和吸收线

    if len(lines) > 0:  # 如果找到的线条数量大于0

        emissionlines_1 = lines[lines['line_type'] == 'emission']  # 获取发射线列表

        abslines_1 = lines[lines['line_type'] == 'absorption']  # 获取吸收线列表

        print("Number of emission lines found:", len(emissionlines_1))  # 打印找到的发射线数量

        print("Number of absorption lines found:", len(abslines_1))  # 打印找到的吸收线数量

    else:  # 如果没有找到线条

        emissionlines_1 = []  # 初始化发射线列表为空

        abslines_1 = []  # 初始化吸收线列表为空

        print("No emission lines found!")  # 打印未找到发射线的消息

    lines = find_lines_derivative(spec_contsub_2, flux_threshold=0.020)  # 查找YSO 1的发射和吸收线

    # lines = find_lines_derivative(spec_contsub_2, flux_threshold=0.03)  # 查找YSO 2的发射和吸收线

    # lines = find_lines_derivative(spec_contsub_2, flux_threshold=0.02)  # 查找YSO 3的发射和吸收线

    if len(lines) > 0:  # 如果找到的线条数量大于0

        emissionlines_2 = lines[lines['line_type'] == 'emission']  # 获取发射线列表

        abslines_2 = lines[lines['line_type'] == 'absorption']  # 获取吸收线列表

        print("Number of emission lines found:", len(emissionlines_2))  # 打印找到的发射线数量

        print("Number of absorption lines found:", len(abslines_2))  # 打印找到的吸收线数量

    else:  # 如果没有找到线条

        emissionlines_2 = []  # 初始化发射线列表为空

        abslines_2 = []  # 初始化吸收线列表为空

        print("No emission lines found!")  # 打印未找到发射线的消息

In [ ]:
# 提取大于13微米的发射线

if emissionlines_1:  # 如果存在发射线1

    emissionlines_1['gauss_line_center'] = 0. * u.micron  # 高斯线中心初始化为0微米

    emissionlines_1['gauss_line_amp'] = 0. * u.jansky  # 高斯线幅度初始化为0焦耳/秒/平方米

    emissionlines_1['gauss_line_stddev'] = 0. * u.micron  # 高斯线标准差初始化为0微米

    emissionlines_1['gauss_line_FWHM'] = 0. * u.micron  # 高斯线全宽半高初始化为0微米

    emissionlines_1['gauss_line_area'] = np.log10(1e-20) * u.W / u.m**2  # 高斯线面积初始化为1e-20的对数值（瓦特/平方米）

    emissionlines_1['no_AltRestWav'] = 0  # 可能的线数量初始化为0

    emissionlines_1['RestWav'] = 0.  # 最近实验测量线的静止波长初始化为0

    emissionlines_1['diff_ft_wav'] = 0.  # 初始线波长估计与拟合高斯之间的差异初始化为0

    emissionlines_1['line_suspect'] = 0  # 如果波长位置差异较大则设置为1，初始化为0

    emissionlines_1['line'] = "                 "  # 用于存储线名称的字符串初始化

    # 遍历光谱中找到的所有发射线

    for idx, emlines in enumerate(emissionlines_1):  # 对于每个发射线

        # 查看原始平滑光谱中找到的线周围的区域

        sw_line = emissionlines_1["line_center"][idx].value - 0.001  # 线的下边界

        lw_line = emissionlines_1["line_center"][idx].value + 0.001  # 线的上边界

        line_region = SpectralRegion(sw_line * u.um, lw_line * u.um)  # 创建光谱区域

        # 线中心变量

        line_cnr = emissionlines_1["line_center"][idx]  # 获取当前线的中心

print(emissionlines_1)  # 打印发射线1的信息

# 提取小于13微米的发射线

if emissionlines_2:  # 如果存在发射线2

    emissionlines_2['gauss_line_center'] = 0. * u.micron  # 高斯线中心初始化为0微米

    emissionlines_2['gauss_line_amp'] = 0. * u.jansky  # 高斯线幅度初始化为0焦耳/秒/平方米

    emissionlines_2['gauss_line_stddev'] = 0. * u.micron  # 高斯线标准差初始化为0微米

    emissionlines_2['gauss_line_FWHM'] = 0. * u.micron  # 高斯线全宽半高初始化为0微米

    emissionlines_2['gauss_line_area'] = np.log10(1e-20) * u.W / u.m**2  # 高斯线面积初始化为1e-20的对数值（瓦特/平方米）

    emissionlines_2['no_AltRestWav'] = 0  # 可能的线数量初始化为0

    emissionlines_2['RestWav'] = 0.  # 最近实验测量线的静止波长初始化为0

    emissionlines_2['diff_ft_wav'] = 0.  # 初始线波长估计与拟合高斯之间的差异初始化为0

    emissionlines_2['line_suspect'] = 0  # 如果波长位置差异较大则设置为1，初始化为0

    emissionlines_2['line'] = "                 "  # 用于存储线名称的字符串初始化

    # 遍历光谱中找到的所有发射线

    for idx, emlines2 in enumerate(emissionlines_2):  # 对于每个发射线

        # 查看原始平滑光谱中找到的线周围的区域

        sw_line = emissionlines_2["line_center"][idx].value - 0.01  # 线的下边界

        lw_line = emissionlines_2["line_center"][idx].value + 0.01  # 线的上边界

        line_region = SpectralRegion(sw_line * u.um, lw_line * u.um)  # 创建光谱区域

        # 线中心变量

        line_cnr = emissionlines_2["line_center"][idx]  # 获取当前线的中心

print(emissionlines_2)  # 打印发射线2的信息

In [ ]:
# 提取大于13微米的吸收线

if abslines_1:  # 如果存在吸收线1

    abslines_1['gauss_line_center'] = 0. * u.micron  # 初始化高斯线中心为0微米

    abslines_1['gauss_line_amp'] = 0. * u.jansky  # 初始化高斯线幅度为0焦耳/秒/平方米/赫兹

    abslines_1['gauss_line_stddev'] = 0. * u.micron  # 初始化高斯线标准差为0微米

    abslines_1['gauss_line_FWHM'] = 0. * u.micron  # 初始化高斯线全宽半高为0微米

    abslines_1['gauss_line_area'] = np.log10(1e-20) * u.W / u.m**2  # 初始化高斯线面积为-20的对数值

    abslines_1['no_AltRestWav'] = 0  # 可能的特征线数量初始化为0

    abslines_1['RestWav'] = 0.  # 最近实验测量线的静止波长初始化为0

    abslines_1['diff_ft_wav'] = 0.  # 初始线波长估计与拟合高斯之间的差异初始化为0

    abslines_1['line_suspect'] = 0  # 如果波长位置差异较大则设置为1，初始化为0

    abslines_1['line'] = "                 "  # 用于存储线名称的字符串初始化

    # 遍历光谱中找到的所有吸收线
    for idx, absorplines in enumerate(abslines_1):  # 使用enumerate获取索引和吸收线

        # 查看原始平滑光谱中找到的线周围的区域
        sw_line = abslines_1["line_center"][idx].value - 0.01  # 计算下边界
        lw_line = abslines_1["line_center"][idx].value + 0.01  # 计算上边界
        line_region = SpectralRegion(sw_line * u.um, lw_line * u.um)  # 创建光谱区域对象

        # 线中心的变量
        line_cnr_abs = abslines_1["line_center"][idx]  # 获取当前吸收线的中心

print(abslines_1)  # 打印吸收线1的信息

# 提取小于13微米的吸收线

if abslines_2:  # 如果存在吸收线2

    abslines_2['gauss_line_center'] = 0. * u.micron  # 初始化高斯线中心为0微米

    abslines_2['gauss_line_amp'] = 0. * u.jansky  # 初始化高斯线幅度为0焦耳/秒/平方米/赫兹

    abslines_2['gauss_line_stddev'] = 0. * u.micron  # 初始化高斯线标准差为0微米

    abslines_2['gauss_line_FWHM'] = 0. * u.micron  # 初始化高斯线全宽半高为0微米

    abslines_2['gauss_line_area'] = np.log10(1e-20) * u.W / u.m**2  # 初始化高斯线面积为-20的对数值

    abslines_2['no_AltRestWav'] = 0  # 可能的特征线数量初始化为0

    abslines_2['RestWav'] = 0.  # 最近实验测量线的静止波长初始化为0

    abslines_2['diff_ft_wav'] = 0.  # 初始线波长估计与拟合高斯之间的差异初始化为0

    abslines_2['line_suspect'] = 0  # 如果波长位置差异较大则设置为1，初始化为0

    abslines_2['line'] = "                 "  # 用于存储线名称的字符串初始化

    # 遍历光谱中找到的所有吸收线
    for idx, absorplines2 in enumerate(abslines_2):  # 使用enumerate获取索引和吸收线

        # 查看原始平滑光谱中找到的线周围的区域
        sw_line = abslines_2["line_center"][idx].value - 0.01  # 计算下边界
        lw_line = abslines_2["line_center"][idx].value + 0.01  # 计算上边界
        line_region = SpectralRegion(sw_line * u.um, lw_line * u.um)  # 创建光谱区域对象

        # 线中心的变量
        line_cnr_abs = abslines_2["line_center"][idx]  # 获取当前吸收线的中心

print(abslines_2)  # 打印吸收线2的信息

### 识别发射线和吸收线

PAH发射特征：6.2微米，7.7微米，8.6微米，11.3微米，12.0微米，12.7微米，14.2微米，16.2微米

硅酸盐吸收特征：10.0 微米，18.0 微米，23.0 微米

CO2冰的吸收特征：15.2微米

其他冰特征：CO（4.67微米）、H2O + HCOOH（6微米）、CH3OH（6.89微米）、CH4（7.7微米）

如果这8条发射线和8条吸收线都不存在，那么这就不是一个年轻恒星对象（YSO）。

In [ ]:
# 首次查看PAH发射特征

pah_emission = [6.2, 7.7, 8.6, 11.3, 12.0, 12.7, 14.2, 16.2] * u.micron  # 已知YSO PAH发射线的列表

len_pah_list = len(pah_emission)  # PAH发射线的数量

pah_emission_detected = []  # 存储检测到的发射线的空列表

line_cnr_list_1 = emissionlines_1["line_center"]  # 从第一个光谱提取的发射线列表

len_line_list = len(line_cnr_list_1)  # 第一个光谱的发射线数量

line_cnr_list_2 = emissionlines_2["line_center"]  # 从第二个光谱提取的发射线列表

len_line_list2 = len(line_cnr_list_2)  # 第二个光谱的发射线数量

count_pah1 = 0  # 计数第一个光谱中存在多少PAH发射线

for i in range(0, len_pah_list):  # 遍历所有PAH发射线

    for j in range(0, len_line_list):  # 遍历第一个光谱中的发射线

        # 检查PAH发射线是否在发射线的±0.15微米范围内
        if line_cnr_list_1[j].value - 0.15 < pah_emission.value[i] < line_cnr_list_1[j].value + 0.15:
            count_pah1 += 1  # 如果找到匹配，计数加一

count_pah2 = 0  # 计数第二个光谱中存在多少PAH发射线

for i in range(0, len_pah_list):  # 遍历所有PAH发射线

    for j in range(0, len_line_list2):  # 遍历第二个光谱中的发射线

        # 检查PAH发射线是否在发射线的±0.15微米范围内
        if line_cnr_list_2[j].value - 0.15 < pah_emission.value[i] < line_cnr_list_2[j].value + 0.15:
            count_pah2 += 1  # 如果找到匹配，计数加一

pah_emission_detected = []  # 存储检测到的发射线的空列表

count_pah = 0  # 重置计数

for i in range(0, len_pah_list):  # 遍历所有PAH发射线

    for j in range(0, len_line_list):  # 遍历第一个光谱中的发射线

        # 检查PAH发射线是否在发射线的±0.15微米范围内
        if line_cnr_list_1[j].value - 0.15 < pah_emission.value[i] < line_cnr_list_1[j].value + 0.15:
            pah_emission_detected.append(line_cnr_list_1[j])  # 将检测到的发射线添加到列表中
            count_pah += 1  # 计数加一

    for j in range(0, len_line_list2):  # 遍历第二个光谱中的发射线

        # 检查PAH发射线是否在发射线的±0.15微米范围内
        if line_cnr_list_2[j].value - 0.15 < pah_emission.value[i] < line_cnr_list_2[j].value + 0.15:
            pah_emission_detected.append(line_cnr_list_2[j])  # 将检测到的发射线添加到列表中
            count_pah += 1  # 计数加一

print(f"{count_pah} PAH emission lines detected in spectrum")  # 输出检测到的PAH发射线数量

if count_pah > 0:  # 如果检测到的PAH发射线数量大于0
    print(pah_emission_detected)  # 输出检测到的PAH发射线

In [ ]:
# 查找硅酸盐吸收特征

sil_absorption = [10.0, 18.0, 23.0] * u.micron  # 已知的YSO硅酸盐吸收线列表

len_sil_list = len(sil_absorption)  # 硅酸盐吸收线的数量

sil_absorption_detected = []  # 存储检测到的线的空数组

line_cnr_list_1 = abslines_1["line_center"]  # 从第一个光谱提取的吸收线列表

len_line_list = len(line_cnr_list_1)  # 第一个光谱的吸收线数量

line_cnr_list_2 = abslines_2["line_center"]  # 从第二个光谱提取的吸收线列表

len_line_list2 = len(line_cnr_list_2)  # 第二个光谱的吸收线数量

count_sil1 = 0  # 计数第一个光谱中存在的硅酸盐吸收线数量

for i in range(0, len_sil_list):  # 遍历所有硅酸盐吸收线

    for j in range(0, len_line_list):  # 遍历第一个光谱中的吸收线

        if line_cnr_list_1[j].value - 0.3 < sil_absorption.value[i] < line_cnr_list_1[j].value + 0.3:  # 检查吸收特征是否在允许范围内

            count_sil1 += 1  # 如果找到，计数加一

count_sil2 = 0  # 计数第二个光谱中存在的硅酸盐吸收线数量

for i in range(0, len_sil_list):  # 遍历所有硅酸盐吸收线

    for j in range(0, len_line_list2):  # 遍历第二个光谱中的吸收线

        if line_cnr_list_2[j].value - 0.3 < sil_absorption.value[i] < line_cnr_list_2[j].value + 0.3:  # 检查吸收特征是否在允许范围内

            count_sil2 += 1  # 如果找到，计数加一

            

sil_absorption_detected = []  # 存储检测到的线的空数组

count_sil = 0  # 重置计数

for i in range(0, len_sil_list):  # 遍历所有硅酸盐吸收线

    for j in range(0, len_line_list):  # 遍历第一个光谱中的吸收线

        if line_cnr_list_1[j].value - 0.3 < sil_absorption.value[i] < line_cnr_list_1[j].value + 0.3:  # 检查吸收特征是否在允许范围内

            sil_absorption_detected.append(sil_absorption[i])  # 将检测到的吸收线添加到列表中

            count_sil += 1  # 计数加一

    for j in range(0, len_line_list2):  # 遍历第二个光谱中的吸收线

        if line_cnr_list_2[j].value - 0.3 < sil_absorption.value[i] < line_cnr_list_2[j].value + 0.3:  # 检查吸收特征是否在允许范围内

            sil_absorption_detected.append(sil_absorption[i])  # 将检测到的吸收线添加到列表中

            count_sil += 1  # 计数加一

            

# 移除相同线的多重检测

drop_dups_sil = list(set(sil_absorption_detected))  # 使用集合去重

print(f"{len(drop_dups_sil)} silicate absorption lines detected in spectrum")  # 打印检测到的硅酸盐吸收线数量

In [ ]:
# 寻找冰的吸收特征

ice_absorption = [4.67, 6.0, 6.9, 7.7, 15.2] * u.micron  # 已知的YSO冰吸收线列表

len_ice_list = len(ice_absorption)  # 冰吸收线数量

ice_absorption_detected = []  # 用于存储检测到的吸收线的空数组

line_cnr_list_1 = abslines_1["line_center"]  # 从第一个光谱提取的吸收线列表

len_line_list = len(line_cnr_list_1)  # 第一个光谱的吸收线数量

line_cnr_list_2 = abslines_2["line_center"]  # 从第二个光谱提取的吸收线列表

len_line_list2 = len(line_cnr_list_2)  # 第二个光谱的吸收线数量

count_ice1 = 0  # 计数第一个光谱中存在多少冰吸收线

for i in range(0, len_ice_list):  # 遍历所有冰吸收线

    for j in range(0, len_line_list):  # 遍历第一个光谱的吸收线

        # 检查冰吸收线是否在光谱吸收线的±0.2微米范围内
        if line_cnr_list_1[j].value - 0.2 < ice_absorption.value[i] < line_cnr_list_1[j].value + 0.2:   
            count_ice1 += 1  # 如果找到匹配，计数加一

count_ice2 = 0  # 计数第二个光谱中存在多少冰吸收线

for i in range(0, len_ice_list):  # 遍历所有冰吸收线

    for j in range(0, len_line_list2):  # 遍历第二个光谱的吸收线

        # 检查冰吸收线是否在光谱吸收线的±0.2微米范围内
        if line_cnr_list_2[j].value - 0.2 < ice_absorption.value[i] < line_cnr_list_2[j].value + 0.2: 
            count_ice2 += 1  # 如果找到匹配，计数加一
            

ice_absorption_detected = []  # 用于存储检测到的吸收线的空数组

count_ice = 0  # 重置计数

for i in range(0, len_ice_list):  # 遍历所有冰吸收线

    for j in range(0, len_line_list):  # 遍历第一个光谱的吸收线

        # 检查冰吸收线是否在光谱吸收线的±0.2微米范围内
        if line_cnr_list_1[j].value - 0.2 < ice_absorption.value[i] < line_cnr_list_1[j].value + 0.2:
            ice_absorption_detected.append(ice_absorption[i])  # 添加检测到的吸收线
            count_ice += 1  # 计数加一

    for j in range(0, len_line_list2):  # 遍历第二个光谱的吸收线

        # 检查冰吸收线是否在光谱吸收线的±0.2微米范围内
        if line_cnr_list_2[j].value - 0.2 < ice_absorption.value[i] < line_cnr_list_2[j].value + 0.2:
            ice_absorption_detected.append(ice_absorption[i])  # 添加检测到的吸收线
            count_ice += 1  # 计数加一
             

# 移除重复检测的相同吸收线

drop_dups_ice = list(set(ice_absorption_detected))  # 使用集合去重

print(f"{len(drop_dups_ice)} ice absorption lines detected in spectrum")  # 打印检测到的冰吸收线数量

### 根据检测到的谱线对YSO进行分类

如果出现多个分类，用户需要仔细观察，YSO可能处于两者之间。例如，如果你得到“这是一个Class 2 YSO”和“这是一个Class 3 YSO”，那么原恒星可能处于这两个阶段之间的演化过程，成为一个Class 2/3 YSO。

In [ ]:
# 如果没有检测到PAH、硅酸盐和冰的吸收线，则打印“这不是一个YSO”
if not pah_emission_detected and not sil_absorption_detected and not ice_absorption_detected:
    print("This is not a YSO.")  # 打印不是YSO的信息

else:
    # 否则，判断是否为YSO 1（最年轻、最嵌入的YSO，具有15.2微米的CO2吸收特征）
    if drop_dups_ice and not drop_dups_sil:
        for i in range(0, count_ice):  # 遍历冰吸收检测的计数
            if ice_absorption_detected[i].value < 15.4 and ice_absorption_detected[i].value > 15.0:
                print("This is a Class 1 YSO.")  # 打印YSO 1的分类

    # 否则，判断是否为YSO 2（更成熟的YSO，具有其他冰吸收和硅酸盐吸收特征）
    if drop_dups_sil:
        print("This is a Class 2 YSO.")  # 打印YSO 2的分类

    elif count_pah == 0 and drop_dups_ice:  # 如果没有PAH且有冰的特征
        for i in range(0, count_ice):  # 遍历冰吸收检测的计数
            if ((ice_absorption_detected[i].value < 4.87 and ice_absorption_detected[i].value > 4.47)
                    or (ice_absorption_detected[i].value < 5.8 and ice_absorption_detected[i].value > 6.2)
                    or (ice_absorption_detected[i].value < 6.69 and ice_absorption_detected[i].value > 7.09)
                    or (ice_absorption_detected[i].value < 7.5 and ice_absorption_detected[i].value > 7.7)):
                print("This is a Class 2 YSO.")  # 打印YSO 2的分类

    # 否则，如果检测到PAH发射特征，则为YSO 3
    if (count_pah > 0 and not drop_dups_ice) or (count_pah > 0 and drop_dups_sil):
        print("This is a Class 3 YSO.")  # 打印YSO 3的分类

### 绘制光谱并标记发射和吸收特征

In [ ]:
fig = plt.figure(figsize=(10, 6))  # 创建一个图形，设置大小为10x6英寸

plt.plot(spec_contsub_1.spectral_axis, spec_contsub_1.flux, color='black')  # 绘制第一个光谱的连续谱，颜色为黑色

plt.plot(spec_contsub_2.spectral_axis, spec_contsub_2.flux, color='black')  # 绘制第二个光谱的连续谱，颜色为黑色

plt.xlabel("Wavelength ({:latex})".format(spec.spectral_axis.unit))  # 设置x轴标签为波长，单位为光谱轴的单位

plt.ylabel("Flux ({:latex})".format(spec.flux.unit))  # 设置y轴标签为通量，单位为光谱的单位

plt.axhline(y=0.0, color='r', linestyle='-')  # 绘制y=0的水平线，颜色为红色

max_emission_plot_y = np.max(spec_contsub_2.flux.value) * 4  # 计算最大发射绘图y值，取第二个光谱通量的最大值的四倍

if len(pah_emission_detected) > 0:  # 如果检测到PAH发射线
    pah_vals = [v.value for v in pah_emission_detected]  # 提取PAH发射线的值
    plt.vlines(pah_vals, -max_emission_plot_y, max_emission_plot_y,  # 绘制PAH发射线的垂直线
               color='r', ls='--', label='PAH emission')  # 颜色为红色，线型为虚线，标签为'PAH emission'
    print("PAH emission lines detected: ", pah_emission_detected)  # 打印检测到的PAH发射线

if len(sil_absorption_detected) > 0:  # 如果检测到硅吸收线
    sil_vals = [v.value for v in sil_absorption_detected]  # 提取硅吸收线的值
    plt.vlines(sil_vals, -max_emission_plot_y, max_emission_plot_y,  # 绘制硅吸收线的垂直线
               color='b', ls='--', label='silicate absorption')  # 颜色为蓝色，线型为虚线，标签为'silicate absorption'
    print("silicate absorption lines detected: ", sil_absorption_detected)  # 打印检测到的硅吸收线

if len(ice_absorption_detected) > 0:  # 如果检测到冰吸收线
    ice_vals = [v.value for v in ice_absorption_detected]  # 提取冰吸收线的值
    plt.vlines(ice_vals, -max_emission_plot_y, max_emission_plot_y,  # 绘制冰吸收线的垂直线
               color='g', ls='--', label='ice absorption')  # 颜色为绿色，线型为虚线，标签为'ice absorption'
    print("ice absorption lines detected: ", ice_absorption_detected)  # 打印检测到的冰吸收线

plt.xlim(5, 35)  # 设置x轴范围为5到35

plt.legend(frameon=False, fontsize='medium')  # 显示图例，设置字体大小为中等，边框关闭

plt.tight_layout()  # 自动调整子图参数，使之填充整个图像区域

plt.show()  # 显示绘制的图形

plt.close()  # 关闭当前图形

In [ ]:
看起来您没有提供任何代码或具体内容。如果您能提供一些JWST望远镜数据处理的Python代码，我将很乐意为您添加中文注释并保持代码结构不变。请将代码粘贴在这里。